In [1]:
%run common.ipynb

[tokenizer.decode(i) for i in tokenizer.get_batch_data(prefix=True)[1]][:10]

['S小写:2796=一一一八五一一八五一一八五一一八四E',
 'S字母:1134=rteyrteyrteyrteyEP',
 'S数字:7796=31187118711871184E',
 'S数字:5099=20398039803980396E',
 'S小写:8386=三三五四七三五四七三五四七三五四四E',
 'S字母:7287=woqtpoqtpoqtpoqriE',
 'S大写:3033=壹贰壹叁叁贰壹叁叁贰壹叁叁贰壹叁贰E',
 'S小写:1955=七八二〇七八二〇七八二〇七八二〇EP',
 'S数字:8287=33151315131513148E',
 'S数字:2781=11125112511251124E']

In [2]:
model_ppo = torch.load('ppo.model')
model_ppo.to(device)
model_ppo.eval()

/root/anaconda3/envs/pt2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModelPPO(
  (model_gen): ModelGEN(
    (feature): GPT2Model(
      (wte): Embedding(53, 64)
      (wpe): Embedding(128, 64)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-3): 4 x GPT2Block(
          (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (fc_out): Linear(in_features=64, out_features=53, bias=False)
  )
  (v_head): Sequential(
    (0): Dropout(p=0.1, inplace=False)

In [3]:
#随机一批数据
_, input_ids, _ = tokenizer.get_batch_data(prefix=True)

#切分成question和answer
split = [i.index(tokenizer.encoder['=']) + 1 for i in input_ids]
question = [input_ids[i][:split[i]] for i in range(len(input_ids))]
answer = [input_ids[i][split[i]:] for i in range(len(input_ids))]

#根据question生成predict
input_ids = [torch.LongTensor(i).unsqueeze(0).to(device) for i in question]
predict = [generate(model_ppo.model_gen, i) for i in input_ids]

#裁剪,只要生成的部分
predict = [p[0].tolist()[len(q):] for p, q in zip(predict, question)]

#解码成文本
question = [tokenizer.decode(i) for i in question]
answer = [tokenizer.decode(i) for i in answer]
predict = [tokenizer.decode(i) for i in predict]

acc = 0
for q, a, p in zip(question, answer, predict):
    print(q, a, p, a == p)
    if a == p:
        acc += 1

acc / len(question)

S数字:6493= 25974597459745972E 25974597459745972E True
S字母:5837= weetpeetpeetpeeriE weetpeetpeetpeeriE True
S小写:4918= 一九六七三九六七三九六七三九六七二E 一九六七三九六七三九六七三九六七二E True
S字母:7622= eproqproqproqpriiE eproqproqproqpriiE True
S数字:7891= 31567156715671564E 31567156715671564E True
S字母:6339= wtetitetitetitetyE wtetitetitetitetyE True
S数字:4440= 17761776177617760E 17761776177617760E True
S大写:8550= 叁肆贰零叁肆贰零叁肆贰零叁肆贰零零E 叁肆贰零叁肆贰零叁肆贰零叁肆贰零零E True
S字母:7275= woqpwoqpwoqpwoqppE woqpwoqpwoqpwoqppE True
S数字:4742= 18969896989698968E 18969896989698968E True
S小写:7644= 三〇五七九〇五七九〇五七九〇五七六E 三〇五七九〇五七九〇五七九〇五七六E True
S大写:2652= 壹零陆零玖零陆零玖零陆零玖零陆零捌E 壹零陆零玖零陆零玖零陆零玖零陆零捌E True
S数字:7386= 29546954695469544E 29546954695469544E True
S字母:8335= eeereeereeereeerpE eeereeereeereeerpE True
S大写:5233= 贰零玖叁肆零玖叁肆零玖叁肆零玖叁贰E 贰零玖叁肆零玖叁肆零玖叁肆零玖叁贰E True
S字母:5038= wpqtrpqtrpqtrpqtwE wpqtrpqtrpqtrpqtwE True
S大写:4874= 壹玖肆玖柒玖肆玖柒玖肆玖柒玖肆玖陆E 壹玖肆玖柒玖肆玖柒玖肆玖柒玖肆玖陆E True
S大写:1109= 肆肆叁陆肆肆叁陆肆肆叁陆肆肆叁陆EP 肆肆叁陆肆肆叁陆肆肆叁陆肆肆叁陆E False
S数字:8917= 35671567156715668E 35671567156715668

0.84375